# Iterations, Generations, and Classic Coroutines

In [1]:
class Spam:
    def __getitem__(self, i):
        print('->', i)
        raise IndexError()


If a class provides `__getitem__`, the `iter()` built-in accepts an instance of that class as iterable and builds an iterator from the instance. 

Python' s iteration machinery will call `__getitem__` with indexes starting from 0. 

In [2]:
spam_can = Spam()
iter(spam_can)

In [3]:
list(spam_can)

-> 0


[]

In [4]:
from collections import abc

isinstance(spam_can, abc.Iterable)

False

In [5]:
class GooseSpam:
    def __iter__(self):
        pass

issubclass(GooseSpam, abc.Iterable)

True

## Using `iter` with a Callable

In [6]:
from random import randint

def d6():
    return randint(1, 6)

d6_iter = iter(d6, 1)

for roll in d6_iter:
    print(roll)

2
4


## Iterable VS Iterator

In [7]:
s = 'ABC'
for char in s:
    print(char)

A
B
C


If there was no `for` statement and we had to emulate the `for` machinery by hand with a `while` loop. 

In [8]:
it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

A
B
C


Python's standard interface for an iterator has two methods: `__next__` and `__iter__`. 

`__iter__` method just returns itself. 

In [9]:
from collections import abc
from abc import abstractmethod


class Iterator(abc.Iterable):

    __slots__ = ()

    @abstractmethod
    def __next__(self):
        'Return the next item from the iterator. When exhausted, raise StopIteration. '
        raise StopIteration
    
    def __iter__(self):
        return self

    @classmethod
    def __subclasshook__(cls, C): 
        if cls is Iterator:
            return _check_methods(C, '__iter__', '__next__')
        return NotImplemented


## `Sentence` Classes with `__iter__`

In [10]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):  
        return SentenceIterator(self.words)  


class SentenceIterator:

    def __init__(self, words):
        self.words = words  
        self.index = 0  

    def __next__(self):
        try:
            word = self.words[self.index]  
        except IndexError:
            raise StopIteration()  
        self.index += 1  
        return word  

    def __iter__(self):  
        return self


In [11]:
s = Sentence('Life of Brian')
it = iter(s)
next(it)

'Life'

In [12]:
for c in it:
    print(c)

of
Brian


**Iterables** have an `__iter__` method that instantiates a new iterator every time. 

**Iterators** implement a `__next__` method that returns individual items, and an `__iter__` method returns itself `self`. 

Therefore, iterators are also iterables, but iterables are not iterators. 

### #`Sentence` takes a Generator Function 

No need for a separate iterator class! 

In [13]:
class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):  
        for word in self.words:
            yield word

In [14]:
s = Sentence('Life of Brian')
it = iter(s)
next(it)

'Life'

In [15]:
for c in it:
    print(c)

of
Brian


##### How Generator Works

In [16]:
def gen_123():
    yield 1
    yield 2
    yield 3

for i in gen_123():
    print(i)

1
2
3


In [17]:
g = gen_123()
next(g)

1

In [18]:
next(g)

2

In [19]:
next(g)

3

In [20]:
try: 
    next(g)
except StopIteration:
    print(StopIteration)

<class 'StopIteration'>


In [21]:
def gen_AB():
    print('Start')
    yield 'A'
    print('Continue')
    yield 'B'
    print('End')

for c in gen_AB():  # equals to g = iter(gen_AB())
    print('->', c)  # The loop prints -> and the value returned by next(g)

Start
-> A
Continue
-> B
End


When it prints 'End', iteration continues with a third call to `next(g)`, while `g = iter(gen_AB())`, advancing to the end of the body of the function, the text 'End'. 


### #`Sentence` takes a Lazy Generator

In [22]:
class Sentence:

    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):  
        for match in RE_WORD.finditer(self.text): 
            yield match.group()

Here, `finditer` builds an iterator over the matches of `RE_WORD` on `self.text`, yeilding `MatchObject` instances. 

And the `match.group()` extracts the matched text from the `MatchObject` instance. 

### #`Sentence` takes Lazy Generator Expression

In [23]:
def gen_AB():
    print('Start')
    yield 'A'
    print('Continue')
    yield 'B'
    print('End')

In [24]:
>>> res1 = [x*3 for x in gen_AB()]

Start
Continue
End


The list comprehension eagerly iterates over the item yielded by the generator object returned by `gen_AB()`: 'A' and 'B'. 

Note the output in the next lines: 'Start', 'Continue' and 'End'. 

In [25]:
for i in res1:
    print('-->', i)

--> AAA
--> BBB


This `for` loop iterates over the `res1` list built by the list comprehension. 

In [26]:
>>> res2 = (x*3 for x in gen_AB())
>>> res2

<generator object <genexpr> at 0x000001B607E20740>

The generator expression returns `res2`, a generator object. 

In [27]:
for i in res2:
    print('-->', i)

Start
--> AAA
Continue
--> BBB
End


Only when the `for` loop iterates over `res2`, this generator gets items from `gen_AB()`. Each iteration of the `for` loop implicitly calls `next(res2)`, which in turn calls `next()` on the generator object returned by `gen_AB()`, advancing it to the next `yield`. 

In [28]:
class Sentence:

    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):  
        return (match.group() for match in RE_WORD.finditer(self.text))

## An Arithmetic Progression Generator

In [29]:
class ArithmeticProgression:

    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end

    def __iter__(self):
        res_type = type(self.begin + self.step)
        res = res_type(self.begin)
        forever = self.end is None
        index = 0
        while forever or res < self.end:
            yield result
            index += 1
            res = self.begin + self.step * index


### `itertools`

In [30]:
import itertools

gen = itertools.count(1, .5)

In [31]:
next(gen)

1

In [32]:
next(gen)

1.5

In [33]:
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

In [34]:
def aritprog(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.cout(first, step)
    if end is None:
        return ap_gen
    return itertools.takewhile(lambda n: n < end, ap_gen)


## Generator Functions in the Standard Library

In [35]:
def vowel(c : str):
    return c.lower() in 'aeious'

list(filter(vowel, 'Aardvark'))

['A', 'a', 'a']

In [36]:
import itertools

list(itertools.filterfalse(vowel, 'Aardvark'))

['r', 'd', 'v', 'r', 'k']

In [37]:
list(itertools.dropwhile(vowel, 'Aardvark'))

['r', 'd', 'v', 'a', 'r', 'k']

In [38]:
list(itertools.takewhile(vowel, 'Aardvark'))

['A', 'a']

In [39]:
list(itertools.compress('Aardvark', (1, 0, 1, 1, 0, 1)))

['A', 'r', 'd', 'a']

In [40]:
list(itertools.islice('Aardvark', 4))

['A', 'a', 'r', 'd']

In [41]:
list(itertools.islice('Aardvark', 4, 7))

['v', 'a', 'r']

In [42]:
list(itertools.islice('Aardvark', 1, 7, 2))

['a', 'd', 'a']

-------------------------------------------------------------------------------------------------------------------------------------------------------

In [43]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]

list(itertools.accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [44]:
list(itertools.accumulate(sample, min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [45]:
list(itertools.accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [46]:
import operator

list(itertools.accumulate(sample, operator.mul))

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

-------------------------------------------------------------------------------------------------------------------------------------------------------

In [47]:
list(enumerate('aldfsfsfdd'))

[(0, 'a'),
 (1, 'l'),
 (2, 'd'),
 (3, 'f'),
 (4, 's'),
 (5, 'f'),
 (6, 's'),
 (7, 'f'),
 (8, 'd'),
 (9, 'd')]

In [48]:
list(enumerate('aldfsfsfdd', 1))

[(1, 'a'),
 (2, 'l'),
 (3, 'd'),
 (4, 'f'),
 (5, 's'),
 (6, 'f'),
 (7, 's'),
 (8, 'f'),
 (9, 'd'),
 (10, 'd')]

In [49]:
list(map(operator.mul, range(11), range(4)))

[0, 1, 4, 9]

-------------------------------------------------------------------------------------------------------------------------------------------------------

In [50]:
list(itertools.chain(enumerate('alddd', 1)))

[(1, 'a'), (2, 'l'), (3, 'd'), (4, 'd'), (5, 'd')]

In [51]:
list(itertools.chain.from_iterable(enumerate('alddd', 1)))

[1, 'a', 2, 'l', 3, 'd', 4, 'd', 5, 'd']

In [52]:
list(itertools.zip_longest(range(7), range(4)))

[(0, 0), (1, 1), (2, 2), (3, 3), (4, None), (5, None), (6, None)]

-------------------------------------------------------------------------------------------------------------------------------------------------------

In [53]:
list(itertools.zip_longest(range(7), range(4), fillvalue='?'))

[(0, 0), (1, 1), (2, 2), (3, 3), (4, '?'), (5, '?'), (6, '?')]

In [54]:
list(itertools.product('ABC', range(2)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

-------------------------------------------------------------------------------------------------------------------------------------------------------

In [55]:
ct = itertools.count()

next(ct)

0

In [56]:
next(ct), next(ct), next(ct)

(1, 2, 3)

In [57]:
list(itertools.repeat(8, 4))

[8, 8, 8, 8]

-------------------------------------------------------------------------------------------------------------------------------------------------------

In [58]:
list(itertools.combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [59]:
list(itertools.combinations_with_replacement('ABC', 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

-------------------------------------------------------------------------------------------------------------------------------------------------------

In [60]:
list(itertools.groupby('LLLLAAGGG'))

[('L', <itertools._grouper at 0x1b607d8b190>),
 ('A', <itertools._grouper at 0x1b607ce4f10>),
 ('G', <itertools._grouper at 0x1b607ce4b50>)]

In [61]:
for char, group in itertools.groupby('LLLLAAGGG'):
    print(char, '-->', list(group))

L --> ['L', 'L', 'L', 'L']
A --> ['A', 'A']
G --> ['G', 'G', 'G']


In [62]:
animals = ['rat', 'duck', 'shark', 'bear', 'dophin', 'lion', 'bat', 'eagle']
for length, group in itertools.groupby(animals, len):
    print(length, '-->', list(group))

3 --> ['rat']
4 --> ['duck']
5 --> ['shark']
4 --> ['bear']
6 --> ['dophin']
4 --> ['lion']
3 --> ['bat']
5 --> ['eagle']


In [63]:
animals = ['rat', 'duck', 'shark', 'bear', 'dophin', 'lion', 'bat', 'eagle']
animals.sort(key=len)
for length, group in itertools.groupby(animals, len):
    print(length, '-->', list(group))

3 --> ['rat', 'bat']
4 --> ['duck', 'bear', 'lion']
5 --> ['shark', 'eagle']
6 --> ['dophin']


-------------------------------------------------------------------------------------------------------------------------------------------------------

In [64]:
list(itertools.tee('ABC'))

[<itertools._tee at 0x1b607e39bc0>, <itertools._tee at 0x1b607e39740>]

In [65]:
list(zip(*itertools.tee('ABC')))

[('A', 'A'), ('B', 'B'), ('C', 'C')]

-------------------------------------------------------------------------------------------------------------------------------------------------------

In [66]:
all([1, 2, 3])

True

In [67]:
all([1, 0, 2])

False

In [68]:
all([])

True

In [69]:
any([1, 2, 3])

True

In [70]:
any([1, 0, 2])

True

In [71]:
any([])

False

In [72]:
g = (n for n in [0, 0.0, 7, 8])
any(g)

True

`any` iterabed over `g` until yielded `7`; then `any` stopped and returned `True`. 

In [73]:
next(g)

8

That's why `8` was still remaining. 

## Subgenerator with `yield from`

In [74]:
def sub_gen():
    yield 1.1
    yield 1.2

def gen():
    yield 1
    for i in sub_gen():
        yield i
    yield 2

print(gen())
for x in gen():
    print(x)

<generator object gen at 0x000001B607E235A0>
1
1.1
1.2
2


In [75]:
def sub_gen():
    yield 1.1
    yield 1.2

def gen():
    yield 1
    yield from sub_gen()
    yield 2

print(gen())
for x in gen():
    print(x)

<generator object gen at 0x000001B607E23BC0>
1
1.1
1.2
2


`yield from` pausees `gen`, and `sub_gen` tackes over until it is exhausted. 

In [76]:
def sub_gen():
    yield 1.1
    yield 1.2
    return 'Done'

def gen():
    yield 1
    result = yield from sub_gen()
    print('-->', result)
    yield 2

for x in gen():
    print(x)

1
1.1
1.2
--> Done
2


### Reinventing chain

In [77]:
def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i

s = 'ABC'
r = range(3)
list(chain(s, r))

['A', 'B', 'C', 0, 1, 2]

In [78]:
def chain(*iterables):
    for i in iterables:
        yield from i

s = 'ABC'
r = range(3)
list(chain(s, r))

['A', 'B', 'C', 0, 1, 2]

### Traversing a Tree

In [79]:
def tree(cls, level=0):
    yield cls.__name__, level
    for sub_cls in cls.__subclasses__():
        yield from tree(sub_cls, level+1)

def display(cls):
    for cls_name, level in tree(cls):
        indent = '\t' * level
        print(f'{indent}{cls_name}')

display(BaseException)

BaseException
	Exception
		TypeError
			FloatOperation
			MultipartConversionError
		StopAsyncIteration
		StopIteration
		ImportError
			ModuleNotFoundError
				PackageNotFoundError
			ZipImportError
		OSError
			ConnectionError
				BrokenPipeError
				ConnectionAbortedError
				ConnectionRefusedError
				ConnectionResetError
					RemoteDisconnected
			BlockingIOError
			ChildProcessError
			FileExistsError
			FileNotFoundError
			IsADirectoryError
			NotADirectoryError
			InterruptedError
				InterruptedSystemCall
			PermissionError
			ProcessLookupError
			TimeoutError
			UnsupportedOperation
			herror
			gaierror
			SSLError
				SSLCertVerificationError
				SSLZeroReturnError
				SSLWantWriteError
				SSLWantReadError
				SSLSyscallError
				SSLEOFError
			Error
				SameFileError
			SpecialFileError
			ExecError
			ReadError
			URLError
				HTTPError
				ContentTooShortError
			BadGzipFile
		EOFError
			IncompleteReadError
		RuntimeError
			RecursionError
			NotImplementedError
				

In [80]:
from keyword import kwlist
from collections.abc import Iterator

short_kw = (k for k in kwlist if len(k) < 5)                # type: abc.Generator

long_kw : Iterator[str] = (k for k in kwlist if len(k) < 5) # type: abc.Iterator

## Coroutine

In [94]:
from collections.abc import Generator

def averager() -> Generator[float, float, None]:
    total = 0.0
    count = 0
    average = 0.0
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count
    

In [95]:
coro_avg = averager()
next(coro_avg)

0.0

Create the coroutine object and start it. 

In [96]:
coro_avg.send(10)

10.0

In [97]:
coro_avg.send(30)

20.0

In [98]:
coro_avg.send(5)

15.0

In [99]:
coro_avg.close()